<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך השלמה לנתוני תהליכי קדסטר 🗺️ 

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
from configs import CNFG
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
import subprocess

<h1 style="text-align:right;font-size:100%">
הכנסת כל התהליכים הקדסטריים בסטאטוסים שרלונטיים לעדכון בנק"ל אל טבלת גבולות רישומי קדסטר

In [ ]:
subprocess.call([CNFG.BatchPath + "CadasterRecordsBorders.bat"]);

In [ ]:
print('    11.c Exporting GUIDs table as CSV file')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

<h1 style="text-align:right;font-size:100%">
טבלת תהליכי קדסטר

In [ ]:
Processes_cols = ['ProcessType', 'ProcessName', 'GlobalID', 'DataSource', 'Status']

Processes = daskgpd.read_file(CNFG.ProductPath, layer = 'CadasterProcessBorders', columns = Processes_cols, npartitions = 8)\
                   .rename(columns = {'GlobalID' : 'ProcessGUID'})\
                   .compute()

del Processes_cols

<h1 style="text-align:right;font-size:100%">
טבלת רישומי קדסטר

In [ ]:
Records_cols = ['Name', 'GlobalID']

Records = daskgpd.read_file(CNFG.ProductPath, layer = 'CadasterRecordsBorders', columns = Records_cols, npartitions = 8)\
                 .rename(columns = {'GlobalID' : 'RecordGUID', 'Name' : 'ProcessName'})\
                 .compute()

del Records_cols

<h1 style="text-align:right;font-size:100%">
בדיקת מזהה ייחודי

In [ ]:
if Processes['ProcessGUID'].is_unique == False: print('Warning: Processes GlobalID is not unique')      
if Records['RecordGUID'].is_unique == False   : print('Warning: Records GlobalID is not unique')

<h1 style="text-align:right;font-size:100%">
מיזוג הטבלאות לקבלת מזהים של שני המקורות לאותה תכנית

In [ ]:
merge = Processes.merge(Records, on = 'ProcessName', how = 'left')
del [Processes, Records]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

<h1 style="text-align:right;font-size:100%">
שמירה כקובץ טבלאי לשימוש מאוחר יותר בהסבה

In [ ]:
merge.to_csv(CNFG.CSVPath + 'ProcessesAndRecords.csv', index = False)